In [1]:
from xgboost import XGBRegressor
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
df=pd.read_csv("Bruska_data.csv")
df.head()

,time stamp,Bruska_wind speed,Bruska_wind direction,Bruska_air density,Zelengrad_wind speed,Zelengrad_wind direction,Zelengrad_air density,Bruska_active power output,Bruska_normally operating number
0,2018-01-01 00:00:00,9.363334,172.79999,1.210,10.635238,165.28897,1.207,21.608133,14.0
1,2018-01-01 01:00:00,9.266666,174.99990,1.210,10.261664,166.09566,1.207,21.050860,14.0
2,2018-01-01 02:00:00,9.586667,174.19984,1.209,13.286666,161.92586,1.207,21.257930,14.0
3,2018-01-01 03:00:00,9.320000,167.53835,1.208,14.777143,159.17044,1.206,20.700977,14.0
4,2018-01-01 04:00:00,9.267742,160.91630,1.207,11.447466,151.64941,1.205,20.331597,14.0


In [3]:
df.shape

(17352, 9)

In [4]:
df.isnull().sum()

time stamp                            0
Bruska_wind speed                     0
Bruska_wind direction                 0
Bruska_air density                    0
Zelengrad_wind speed                  0
Zelengrad_wind direction              0
Zelengrad_air density                 0
Bruska_active power output          163
Bruska_normally operating number    163
dtype: int64

In [5]:
df.dropna(inplace=True)

In [6]:
df.drop(['time stamp'], axis = 1, inplace = True) 

In [7]:
df.drop(['Bruska_normally operating number'], axis = 1, inplace = True)

In [8]:
s1= MinMaxScaler(feature_range=(0,1))
X= s1.fit_transform(df)

In [9]:
s2=MinMaxScaler(feature_range=(0,1))
y= s2.fit_transform(df[['Bruska_active power output']])

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42,shuffle=False)

In [11]:
model=XGBRegressor(tree_method='approx',n_estimators=300,max_depth=9,learning_rate=0.1,colsample_bytree=0.9)
import time
start=time.time()
model.fit(X_train,y_train)
end=time.time()
continential= end-start
print("convergence time:",continential)

convergence time: 5.295178651809692


In [12]:
start=time.time()
pred = model.predict(X_test)
end=time.time()
continential=end-start
print("inference time:",continential)
pred



inference time: 0.033586978912353516


array([ 1.1868860e-03,  1.5589096e-07, -5.4866075e-07, ...,
        3.0297473e-02,  4.7964178e-02,  5.2602603e-03], dtype=float32)

In [13]:
pred.shape

(3438,)

In [14]:
prediction = pred.reshape(pred.shape[0], 1)

In [15]:
prediction.shape

(3438, 1)

In [16]:
Pred= s2.inverse_transform(prediction)

In [17]:
Actual= s2.inverse_transform(y_test)

In [18]:
x=Pred
z= Actual

In [19]:
from sklearn.metrics import mean_squared_error
import math

 
MSE = mean_squared_error(z,x)
 
RMSE = math.sqrt(MSE)
from sklearn.metrics import mean_absolute_error
MAE=mean_absolute_error(z,x)
MAE
from sklearn.metrics import mean_absolute_percentage_error
MAPE=mean_absolute_percentage_error(z,x)
MAPE

print("Root Mean Square Error:",RMSE)
print("Mean Square Error:", MSE)
print("Mean Absolute Error:",MAE)
print("Mean Absolute Percentage Error:",MAPE)

Root Mean Square Error: 0.27144176969235817
Mean Square Error: 0.0736806343337192
Mean Absolute Error: 0.09526581029463196
Mean Absolute Percentage Error: 4506451986014.038


In [20]:
max= x.max()
max

35.840973

In [21]:
min=x.min()
min

-0.27086982

In [22]:
NRMSE= (RMSE/(max-min))*100
print("Normalized Root Mean Square Error:",NRMSE)

Normalized Root Mean Square Error: 0.7516696638054574


In [23]:
Calculated =pd.DataFrame(Pred, columns = ['Prediction'])

In [24]:
Actual=pd.DataFrame(Actual, columns=["Actual"])

In [25]:
combined_df = pd.concat([Calculated, Actual], axis=1)

In [26]:
combined_df.to_csv(r'C:\Users\sunrise\Desktop\Wind speed forecasting\Bruska\XgBoost\XgBoost_prediction.csv', index = False)